In [2]:
# decorators
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()
print(target)

running inner()
<function deco.<locals>.inner at 0x105effc70>


In [4]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x105eff9a0>)
running register(<function f2 at 0x105effa30>)
running main()
registry -> [<function f1 at 0x105eff9a0>, <function f2 at 0x105effa30>]
running f1()
running f2()
running f3()


In [5]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3) # throws, doesn't use global b

3


UnboundLocalError: local variable 'b' referenced before assignment

In [22]:
# Python does not have a program global scope, only module global scopes.
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [7]:
b

9

In [8]:
# dis module an easy way to disassemble the bytecode of Python functions
from dis import dis
dis(f1)

 10           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('running f1()')
              4 CALL_FUNCTION            1
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [9]:
dis(f3)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [10]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

avg = Averager()
avg(10)

10.0

In [11]:
avg(11)

10.5

In [12]:
avg(12)

11.0

In [14]:
# using a function for closure instead of class
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
        
    return averager

In [15]:
avg = make_averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(15)

12.0

In [18]:
avg.__code__.co_varnames

('new_value', 'total')

In [19]:
avg.__code__.co_freevars

('series',)

In [20]:
avg.__closure__

(<cell at 0x10618eb90: list object at 0x1065642c0>,)

In [21]:
avg.__closure__[0].cell_contents

[10, 11, 15]

In [23]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total # we have to do this for closure (immutable types) so were not re-assigning here making it a local
        count += 1
        total += new_value
        return total/count

    return averager

In [28]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s {name}({arg_str}) -> {result!r}')
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12460550s snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000092s factorial(1) -> 1
[0.00001704s factorial(2) -> 2
[0.00002846s factorial(3) -> 6
[0.00003942s factorial(4) -> 24
[0.00005067s factorial(5) -> 120
[0.00006304s factorial(6) -> 720
6! = 720


In [29]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000062s fibonacci(0) -> 0
[0.00000058s fibonacci(1) -> 1
[0.00017446s fibonacci(2) -> 1
[0.00000042s fibonacci(1) -> 1
[0.00000062s fibonacci(0) -> 0
[0.00000037s fibonacci(1) -> 1
[0.00002925s fibonacci(2) -> 1
[0.00004904s fibonacci(3) -> 2
[0.00029025s fibonacci(4) -> 3
[0.00000033s fibonacci(1) -> 1
[0.00000033s fibonacci(0) -> 0
[0.00000038s fibonacci(1) -> 1
[0.00002000s fibonacci(2) -> 1
[0.00004042s fibonacci(3) -> 2
[0.00000029s fibonacci(0) -> 0
[0.00000033s fibonacci(1) -> 1
[0.00001958s fibonacci(2) -> 1
[0.00000033s fibonacci(1) -> 1
[0.00000037s fibonacci(0) -> 0
[0.00000029s fibonacci(1) -> 1
[0.00001963s fibonacci(2) -> 1
[0.00003879s fibonacci(3) -> 2
[0.00007721s fibonacci(4) -> 3
[0.00016721s fibonacci(5) -> 5
[0.00048200s fibonacci(6) -> 8
8


In [33]:
# dynamic programming: optimizing recursive calls
import functools

@functools.cache # there is also a @lru_cache(maxsize=2**20) more control of maxsize
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

if __name__ == '__main__':
    print(fibonacci(30))

[0.00000071s fibonacci(0) -> 0
[0.00000062s fibonacci(1) -> 1
[0.00038367s fibonacci(2) -> 1
[0.00000079s fibonacci(3) -> 2
[0.00040508s fibonacci(4) -> 3
[0.00000075s fibonacci(5) -> 5
[0.00042529s fibonacci(6) -> 8
[0.00000075s fibonacci(7) -> 13
[0.00044529s fibonacci(8) -> 21
[0.00000071s fibonacci(9) -> 34
[0.00047762s fibonacci(10) -> 55
[0.00000071s fibonacci(11) -> 89
[0.00049867s fibonacci(12) -> 144
[0.00000071s fibonacci(13) -> 233
[0.00052504s fibonacci(14) -> 377
[0.00000071s fibonacci(15) -> 610
[0.00054475s fibonacci(16) -> 987
[0.00000071s fibonacci(17) -> 1597
[0.00056454s fibonacci(18) -> 2584
[0.00000071s fibonacci(19) -> 4181
[0.00058479s fibonacci(20) -> 6765
[0.00000071s fibonacci(21) -> 10946
[0.00060554s fibonacci(22) -> 17711
[0.00000067s fibonacci(23) -> 28657
[0.00063521s fibonacci(24) -> 46368
[0.00000071s fibonacci(25) -> 75025
[0.00065483s fibonacci(26) -> 121393
[0.00000071s fibonacci(27) -> 196418
[0.00068937s fibonacci(28) -> 317811
[0.00000104s fibonac

In [32]:
"""
@alpha
@beta
def my_fn():
    ...
is the same as:
my_fn = alpha(beta(my_fn))
"""

'\n@alpha\n@beta\ndef my_fn():\n    ...\nis the same as:\nmy_fn = alpha(beta(my_fn))\n'

In [36]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch # base function
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br />\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

In [37]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register' f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)

        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x1064f32e0>)
running register(active=True)->decorate(<function f2 at 0x1064f0820>)


In [38]:
registry

{<function __main__.f2()>}

In [39]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x105eff9a0>)


<function __main__.f3()>

In [40]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [41]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x1064f0820>)


<function __main__.f2()>

In [42]:
registry

{<function __main__.f3()>}

In [55]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals())) #  linters will complain, could write out each local var twice, this is clearner
            return _result
        return clocked
    return decorate
    
# @clock()
# def snooze(seconds):
#     time.sleep(seconds)

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.12807437498122454s
snooze: 0.12722762499470264s
snooze: 0.12785687495488673s


In [56]:
# when using decorators its best to use a class and __call__
# for example purposes functions are easier to understand
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked